# Predicting the Future Exchange Rates by training a Linear Regression Model using the historical data.

## Loading the required libraries and packages

Now, let's load all the required packages to the project file, so that we can efficiently code, without having to bother about calling the packages while coding.

This code imports the necessary libraries for data analysis and visualization such as pandas, numpy, plotly.graph_objs and sklearn.linear_model. It also imports the mean_absolute_percentage_error function from sklearn.metrics and the drive function from google.colab.

The LinearRegression function from sklearn.linear_model is used to perform linear regression on the data. The mean_absolute_percentage_error function from sklearn.metrics is used to calculate the mean absolute percentage error of the model.

The drive function from google.colab is used to mount Google Drive to the Colab notebook.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from google.colab import drive

## Connect to Drive and load the historical data of the Currency Pair (EUR - USD)

Now, lets mount the Google Drive for the project file, so that the downloaded historical data of a specific ticker can be accessed, on which our model can be trained.

In [2]:
drive.mount ('/content/drive')
project_directory = '/content/drive/MyDrive/St Clair/AI Project (DAB106)'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This code reads a CSV file named EURUSD13114.csv located in the project_directory and stores it in a pandas DataFrame called df. The Date column is converted to a datetime format using the pd.to_datetime function. The set_index function is used to set the Date column as the index of the DataFrame.

In [3]:
df = pd.read_csv(project_directory+'/EURUSD13114.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

## Data Cleaning and Pre - Processing

Now, from the raw data pull, we can see that this data, in its current state, cannot be used for training the model. Hence, we need to clean the data and extract the target variable and the features of the dataset that are contributing to the value of the target variable.

This code creates a variable X which is a 2D numpy array containing the Open, High, Low, and Close prices of the df DataFrame, except for the last row. The y variable is also a 2D numpy array containing the Open, High, Low, and Close prices of the df DataFrame, except for the first row. The X variable is used as the input features to predict the y variable.

In [4]:
X = df[['Open', 'High', 'Low', 'Close']].values[:-1]  # Use open, high, low, and close prices as features
y = df[['Open', 'High', 'Low', 'Close']].values[1:]   # Predict the next period's open, high, low, and close prices

### Splitting the data into traning and validation datasets.

This code creates a variable train_size which is 90% of the data of X. The X_train and y_train variables are created by slicing the X and y variables up to the train_size index. The X_test and y_test variables are created by slicing the X and y variables from the train_size index to the end of the array.

In [5]:
train_size = int(0.9 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Building the Model to train the data

This code creates a LinearRegression object called lr. The n_epochs variable is set to 100. The lr.fit function is used to fit the model to the training data. The lr.predict function is used to make predictions on the testing data. The lr.score function is used to calculate the accuracy of the model.

A DataFrame called df_pred is created with the predicted Open, High, Low, and Close prices and corresponding dates. The set_index function is used to set the Date column as the index of the DataFrame. The last_date variable is set to the last date in the df DataFrame. The next_day variable is set to the predicted price for the next day. The predicted price for the next day is added to the df_pred DataFrame. The pd.concat function is used to combine the actual and predicted dataframes.

The accuracy of the model is printed at the end of the code.

In [6]:
lr = LinearRegression()
n_epochs = 100
for epoch in range(n_epochs):
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # Use the model to make predictions on the testing data
    y_pred = lr.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = lr.score(X_test, y_test)

    # Create a dataframe with the predicted open, high, low, and close prices and corresponding dates
    df_pred = pd.DataFrame({'Date': df.index.values[train_size+1:], 'Open': y_pred[:, 0], 'High': y_pred[:, 1], 'Low': y_pred[:, 2], 'Close': y_pred[:, 3]})
    df_pred.set_index('Date', inplace=True)

    # Predict the next day
    last_date = df.index[-1]
    next_day = lr.predict(np.array([df[['Open', 'High', 'Low', 'Close']].values[-1]]).reshape(1, -1))

    # Add predicted price for the next day to the dataframe
    last_date = last_date + pd.DateOffset(days=1)
    df_pred.loc[last_date] = next_day[0]

    # Combine actual and predicted dataframes
    df_combined = pd.concat([df, df_pred])

print(accuracy)

0.996529190227823


## Plotting the Predictions of the model

Now, let's compare the predictions of the model, by plotting them graph comparing them with the actual values, to see the amoout of variation and deviation we have from the actual values. Specifically, we are plotting the predicted "High" and "Low" to see if we can predict the day's trading range.

This code creates a Candlestick trace called trace_actual using the go.Candlestick function. The Scatter traces called trace_predicted_low and trace_predicted_high are created using the go.Scatter function. The data variable is created by combining the trace_actual, trace_predicted_low, and trace_predicted_high traces. The layout variable is created using the go.Layout function. The fig variable is created using the go.Figure function and the data and layout variables. The fig.show function is used to display the plot.

In [7]:
# Create the actual and predicted price traces
trace_actual = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Actual Prices', increasing_line_color='green', decreasing_line_color='red', opacity=0.4)
trace_predicted_low = go.Scatter(x=df_pred.index, y=df_pred['Low'], name='Predicted Day Low', mode='lines', line=dict(color='red'), opacity=0.7)
trace_predicted_high = go.Scatter(x=df_pred.index, y=df_pred['High'], name='Predicted Day High', mode='lines', line=dict(color='green'), opacity=0.7)

# Combine the traces and set the layout
data = [trace_actual, trace_predicted_low, trace_predicted_high]
layout = go.Layout(title='Actual and Predicted Prices over Time', xaxis_rangeslider_visible=False, legend=dict(orientation='h', y=1.1))

# Create the figure and show the plot
fig = go.Figure(data=data, layout=layout)
fig.show()

As you can see above, we have started out with the data points from the 1970s all the way to 2023. We have plotted the predicted day "High" and "Low", so that we can predict the day range and then compare it with the actual price. We can see that the model managed to predict the ranges more or less accurately.

The fallacy here is that, currency pairs are generally influenced by real time events, and are pricing is not completely mathematical. Hence, we can see the deviation from the prediction at some points.